In [11]:
import pandas as pd
import numpy as np
from datetime import datetime

REFERENCE_DATE = pd.to_datetime('2021-01-01') 

def preprocess_basic(df: pd.DataFrame) -> pd.DataFrame:
    """
    Dataclean、transfer data type and create time and age Feature.
    Which is the basic of age Feature.
    
    Args:
        df: "train.csv" DataFrame.
        
    Returns:
        DataFrame: train.csv after preprocessing.
    """
    df = df.copy()

    
    df['trans_num'] = df['trans_num'].astype(str)
    df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
    df['dob'] = pd.to_datetime(df['dob'])
    
    
    df = df.sort_values('trans_date_trans_time').reset_index(drop=True)

    
    df['trans_hour'] = df['trans_date_trans_time'].dt.hour
    df['trans_day'] = df['trans_date_trans_time'].dt.dayofweek # 0 is monday, and 6 is sunday.
    df['trans_month'] = df['trans_date_trans_time'].dt.month
    df['trans_year'] = df['trans_date_trans_time'].dt.year
    
    # Count user age
    # Refernce date is 2021-01-01
    df['age'] = (REFERENCE_DATE - df['dob']).dt.days // 365
    
    
    cols_to_drop = [          
        'first', 'last', 'street', 'dob', 
        'unix_time'        
    ]
    
    df = df.drop(columns=cols_to_drop, errors='ignore')

    return df

In [12]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Distance 
    R=6317
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine 
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # distance
    distance = R * c
    return distance

def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    """
    Distance Metrics、Card-level Rolling Statistics / Temporal Features, and Category/Merchant Amount Aggregations.
    
    """
    df = df.copy()
    
    df['merch_haversine_dist'] = haversine_distance( 
        df['lat'], df['long'], df['merch_lat'], df['merch_long']
    )
    df = df.drop(columns=['lat', 'long', 'merch_lat', 'merch_long'], errors='ignore')
    
    
    # Card-level Rolling Statistics / Temporal Features
    
    # Transfer  time to Unix (seconds)
    df['unix_time_sec'] = df['trans_date_trans_time'].astype(np.int64) // 10**9
    
    # Temporal Feature
    df['cc_time_since_last'] = df.groupby('cc_num')['unix_time_sec'].diff().fillna(999999) 
    
    
    # Cumulative Statistics
    df['cc_count_cum'] = df.groupby('cc_num').cumcount()
    
    df['cc_mean_amt_cum'] = df.groupby('cc_num')['amt'].transform(
        lambda x: x.shift(1).expanding().mean().fillna(0)
    )
    
    first_time = df.groupby('cc_num')['unix_time_sec'].transform('min')
    df['cc_time_diff_total'] = df['unix_time_sec'] - first_time + 1 
    df['cc_freq'] = df['cc_count_cum'] / df['cc_time_diff_total']
    
    
    # Category/Merchant Amount Aggregations
    
    
    
    df['category_mean_amt'] = df.groupby('category')['amt'].transform('mean')
    df['amt_vs_cat_mean'] = df['amt'] / df['category_mean_amt']
    

    df['amt_per_pop'] = df['amt'] / (df['city_pop'] + 1)
    

    df = df.drop(columns=['city', 'state', 'zip', 'unix_time_sec', 'cc_time_diff_total', 'cc_num', 'merchant'], errors='ignore')
    
    
    return df
    
   

In [13]:
train_df = pd.read_csv('train.csv')
test_df =pd.read_csv('test.csv')
df_full = feature_engineering(preprocess_basic(train_df))
df_test_proc = feature_engineering(preprocess_basic(test_df))

TARGET_REG = 'amt'
# Exclude target, classification target, IDs, and original timestamp
FEATURES_REG = [col for col in df_full.columns if col not in [TARGET_REG, 'is_fraud', 'trans_num', 'trans_date_trans_time']]

X_reg = df_full[FEATURES_REG]
y_reg = df_full[TARGET_REG]

# Define Feature Types and Preprocessor ---
# These feature lists must strictly correspond to the output of your feature_engineering function
numeric_features = [
    'city_pop', 'age', 'merch_haversine_dist', 'cc_time_since_last', 
    'cc_mean_amt_cum', 'cc_count_cum', 'cc_freq', 
    'category_mean_amt', 'amt_vs_cat_mean', 'amt_per_pop'
] 
categorical_features = [
    'category', 'gender', 'job', 'trans_hour', 'trans_day', 
    'trans_month', 'trans_year' 
] 


from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor_reg = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='passthrough' 
)

# Task 2 Regression

In [14]:

## ### 1. Model Comparison: Ridge vs. LGBMR (Part I Requirement)

from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

# Cross-validation setup
cv = KFold(n_splits=3, shuffle=True, random_state=42)
SAFE_N_JOBS = 2

# Define RMSE evaluation function (for cross_val_score)
def evaluate_rmse_cv(model, X, y, cv):
    """
    Evaluates the model's RMSE performance using K-Fold cross-validation.
    """
    # cross_val_score 使用 'neg_mean_squared_error' 
    scores = cross_val_score(model, X, y, cv=cv, 
                             scoring='neg_mean_squared_error', n_jobs=SAFE_N_JOBS)
    rmse_scores = np.sqrt(-scores)
    return rmse_scores.mean(), rmse_scores.std()

# Model A: Ridge Regression (Baseline) ---
ridge_model = Pipeline(steps=[
    ('preprocessor', preprocessor_reg),
    ('regressor', Ridge(alpha=10.0, random_state=42)) 
])

# Model B: LightGBM Regressor (High Performance/Efficiency) ---
lgbm_reg_model = Pipeline(steps=[
    ('preprocessor', preprocessor_reg),
    # Reduced n_estimators to 50 to ensure speed
    ('regressor', LGBMRegressor(n_estimators=50, max_depth=5, learning_rate=0.1, random_state=42, n_jobs=1))
    
])

print("--- Comparing Ridge Regression (Baseline) ---")
ridge_rmse_mean, _ = evaluate_rmse_cv(ridge_model, X_reg, y_reg, cv)
print(f"Ridge CV RMSE: {ridge_rmse_mean:.2f}")

print("\n--- Comparing LightGBM Regressor (High Performance/Efficiency) ---")
lgbm_reg_rmse_mean, _ = evaluate_rmse_cv(lgbm_reg_model, X_reg, y_reg, cv)
print(f"LGBMR CV RMSE: {lgbm_reg_rmse_mean:.2f}")

--- Comparing Ridge Regression (Baseline) ---
Ridge CV RMSE: 39.10

--- Comparing LightGBM Regressor (High Performance/Efficiency) ---
LGBMR CV RMSE: 77.46


### 2. Part II Regression Task: Model Selection and Comparison

**Objective:** Predict the transaction amount (`amt`), with the performance metric being $RMSE \le 140$.

To perform efficient and robust model selection, we compared a linear model (Ridge Regression) against a high-performance ensemble tree model (LightGBM Regressor), utilizing **3-Fold Cross-Validation (CV)** as the primary evaluation method.

| Model | Role | CV RMSE Mean | Conclusion |
| :--- | :--- | :--- | :--- |
| **Ridge Regression** | Baseline Model | $39.10$ | **Exceptional performance, far exceeding the $\le 140$ target.** |
| **LightGBM Regressor** | High-Performance Model | $77.46$ | Excellent performance, but inferior to Ridge. |

#### Rationale and Final Choice: Ridge Regression

Based on the cross-validation results, **Ridge Regression** achieved the lowest RMSE of $39.10$, demonstrating performance significantly superior to LightGBM.

* **Technical Insight:** This suggests that the aggregate features constructed during our feature engineering process (such as `category_mean_amt` and `amt_vs_cat_mean`) have an **extremely strong, nearly linear correlation** with the target variable, `amt`.
* **Model Selection:** As a regularized linear model, Ridge is capable of capturing this linear relationship with minimal complexity, maximal speed, and high stability.
* **Conclusion:** We select **Ridge Regression** as the final model for Part II.

In [15]:
## ### 3. Hyperparameter Tuning: Optimizing Ridge Regression's alpha

from sklearn.model_selection import GridSearchCV

print("\n## Ridge Regression Hyperparameter Tuning")

# Tuning the alpha parameter (L2 regularization strength)
param_grid_ridge = {
    'regressor__alpha': [1.0, 10.0, 100.0, 500.0]
}

ridge_grid_search = GridSearchCV(
    ridge_model, 
    param_grid_ridge, 
    cv=cv, # Using 3-fold Cross-Validation
    scoring='neg_mean_squared_error', 
    n_jobs=SAFE_N_JOBS, 
    verbose=1 
)

ridge_grid_search.fit(X_reg, y_reg)

best_rmse_ridge = np.sqrt(-ridge_grid_search.best_score_)
best_alpha = ridge_grid_search.best_params_['regressor__alpha']

print(f"\nBest Ridge Model CV RMSE: {best_rmse_ridge:.2f}")
print(f"Best parameter alpha: {best_alpha}")

# Recording the final parameter
FINAL_REG_ALPHA = best_alpha


## Ridge Regression Hyperparameter Tuning
Fitting 3 folds for each of 4 candidates, totalling 12 fits

Best Ridge Model CV RMSE: 39.19
Best parameter alpha: 500.0


In [ ]:
## ### 4. Final Performance Confirmation (Using Public Validation Set test.csv)

# 1. Create the final model (using the optimal alpha
final_ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_reg),
    ('regressor', Ridge(alpha=FINAL_REG_ALPHA, random_state=42))
])

# 2. Train on the entire training set (train_csv)
final_ridge_pipeline.fit(X_reg, y_reg)

# 3. Predict on test.csv
X_test_reg = df_test_proc.drop(columns=[TARGET_REG, 'is_fraud', 'trans_num', 'trans_date_trans_time'], errors='ignore')
y_test_reg_true = df_test_proc[TARGET_REG] 
test_predictions = final_ridge_pipeline.predict(X_test_reg)
test_predictions[test_predictions < 0] = 0 

# 4. Calculate RMSE on test.csv
final_rmse_on_test = np.sqrt(mean_squared_error(y_test_reg_true, test_predictions))

print(f"\n--- Final Performance Report ---")
print(f"Final Ridge Model RMSE on test.csv: {final_rmse_on_test:.2f}")

# Final conclusion
if final_rmse_on_test <= 140:
    print(f"✅ Part II Goal Achieved: Final model performance ({final_rmse_on_test:.2f}) meets the RMSE <= 140 requirement.")
else:
    print("⚠️ Warning: Final performance did not meet the target.")


--- Final Performance Report ---
Final Ridge Model RMSE on test.csv: 29.93
✅ Part II Goal Achieved: Final model performance (29.93) meets the RMSE <= 140 requirement.


# Task 3

In [45]:

TARGET_CLS = 'is_fraud'
y_train_cls = df_full[TARGET_CLS] 


X_train = X_reg 


SAFE_N_JOBS = 2

print(f"Task 3 Target Variable y_train_cls prepared. Fraud ratio: {y_train_cls.mean():.4%}")

Task 3 Target Variable y_train_cls prepared. Fraud ratio: 0.5220%


In [ ]:
## ### 2. Part III Classification Task: Model Comparison

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier 
from sklearn.model_selection import StratifiedKFold, cross_val_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, make_scorer


# --- Configuration ---
f1_macro_scorer = make_scorer(f1_score, average='macro')
# Using Stratified 3-fold for speed and class balance handling
cv_cls = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) 


def evaluate_f1_macro_cv(model, X, y, cv):
    """
    Evaluates the model's F1-Macro performance using Stratified K-Fold CV.
    """
    scores = cross_val_score(model, X, y, cv=cv, 
                             scoring=f1_macro_scorer, n_jobs=SAFE_N_JOBS)
    return scores.mean(), scores.std()

# --- Model C: Logistic Regression (Baseline) ---
lr_model = Pipeline(steps=[
    ('preprocessor', preprocessor_reg), 
    ('classifier', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42, max_iter=1000))
])

# --- Model D: LightGBM Classifier + SMOTE (High Performance/Efficiency) ---
lgbm_model_smote = ImbPipeline(steps=[
    ('preprocessor', preprocessor_reg), 
    ('smote', SMOTE(sampling_strategy='minority', random_state=42)), 
    ('classifier', LGBMClassifier(n_estimators=200, random_state=42, n_jobs=1, verbose=-1)) 
])

print("--- Comparing Logistic Regression (Baseline) ---")
lr_f1_mean, lr_f1_std = evaluate_f1_macro_cv(lr_model, X_train, y_train_cls, cv_cls)
print(f"LR CV F1-Macro: {lr_f1_mean:.4f} (+/- {lr_f1_std:.4f})")

print("\n--- Comparing LightGBM Classifier + SMOTE (Selected) ---")
lgbm_f1_mean, lgbm_f1_std = evaluate_f1_macro_cv(lgbm_model_smote, X_train, y_train_cls, cv_cls)
print(f"LGBMC + SMOTE CV F1-Macro: {lgbm_f1_mean:.4f} (+/- {lgbm_f1_std:.4f})")

--- Comparing Logistic Regression (Baseline) ---
LR CV F1-Macro: 0.5087 (+/- 0.0014)

--- Comparing LightGBM Classifier + SMOTE (Selected) ---
LGBMC + SMOTE CV F1-Macro: 0.8814 (+/- 0.0048)


In [46]:
## ### 2. Part III Final Model Comparison (LR vs LGBM vs XGBoost)

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier 
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, make_scorer
from sklearn.pipeline import Pipeline # Standard Pipeline for LR

# --- Configuration ---
f1_macro_scorer = make_scorer(f1_score, average='macro')
cv_cls = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) 

# --- Shared Parameters ---
N_ESTIMATORS_HIGH = 200 # N_ESTIMATORS for high performance evaluation
N_ESTIMATORS_RISK = 50  # N_ESTIMATORS for speed risk assessment
SMOTE_SAMPLING_RISK = 0.5 # SMOTE sampling ratio for risk assessment


def evaluate_f1_macro_cv_timed(model, X, y, cv):
    """
    Evaluates the model's F1-Macro performance, standard deviation, and 3-Fold CV total time.
    """
    import time
    start_time = time.time()
    # Perform 3-Fold CV using cross_val_score
    scores = cross_val_score(model, X, y, cv=cv, 
                             scoring=f1_macro_scorer, n_jobs=SAFE_N_JOBS)
    end_time = time.time()
    
    mean_f1 = scores.mean()
    std_f1 = scores.std()
    total_time = end_time - start_time
    single_time = total_time / 3
    
    return mean_f1, std_f1, total_time, single_time


# 1. Model Definition


# 1.1 Model LR: Logistic Regression (Baseline)
lr_model = Pipeline(steps=[
    ('preprocessor', preprocessor_reg), 
    ('classifier', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42, max_iter=1000))
])

# 1.2 Model LGBM-High: LightGBM (n_estimators=200) 
lgbm_high_model = ImbPipeline(steps=[
    ('preprocessor', preprocessor_reg), 
    ('smote', SMOTE(sampling_strategy='minority', random_state=42)), 
    ('classifier', LGBMClassifier(n_estimators=N_ESTIMATORS_HIGH, random_state=42, n_jobs=1, verbose=-1)) 
])

# 1.3 Model XGB-Risk: XGBoost (n_estimators=50) 
xgb_risk_model = ImbPipeline(steps=[
    ('preprocessor', preprocessor_reg), 
    ('smote', SMOTE(sampling_strategy=SMOTE_SAMPLING_RISK, random_state=42)), 
    ('classifier', XGBClassifier(n_estimators=N_ESTIMATORS_RISK, use_label_encoder=False, 
                                 eval_metric='logloss', random_state=42, n_jobs=1)) 
])

# 1.4 Model LGBM-Risk: LightGBM (n_estimators=50) 
lgbm_risk_model = ImbPipeline(steps=[
    ('preprocessor', preprocessor_reg), 
    ('smote', SMOTE(sampling_strategy=SMOTE_SAMPLING_RISK, random_state=42)), 
    ('classifier', LGBMClassifier(n_estimators=N_ESTIMATORS_RISK, random_state=42, n_jobs=1, verbose=-1)) 
])



# 2. Model Execution and Output


print("--- 1. Logistic Regression (Baseline) ---")
lr_f1, lr_std, lr_time, _ = evaluate_f1_macro_cv_timed(lr_model, X_train, y_train_cls, cv_cls)
print(f"✅ LR CV F1-Macro: {lr_f1:.4f} (+/- {lr_std:.4f}) | Time: {lr_time:.2f} s")

print("\n--- 2. LightGBM (N_EST=200) - Final Performance Target ---")
lgbm_high_f1, lgbm_high_std, lgbm_high_time, lgbm_high_single = evaluate_f1_macro_cv_timed(lgbm_high_model, X_train, y_train_cls, cv_cls)
print(f"✅ LGBM CV F1-Macro: {lgbm_high_f1:.4f} (+/- {lgbm_high_std:.4f})")
print(f"   Time: {lgbm_high_time:.2f} s (Single Est: {lgbm_high_single:.2f} s)")

print("\n--- 3. XGBoost (N_EST=50) - Speed Risk Assessment ---")
xgb_risk_f1, xgb_risk_std, xgb_risk_time, xgb_risk_single = evaluate_f1_macro_cv_timed(xgb_risk_model, X_train, y_train_cls, cv_cls)
print(f"✅ XGBoost CV F1-Macro: {xgb_risk_f1:.4f} (+/- {xgb_risk_std:.4f})")
print(f"   Time: {xgb_risk_time:.2f} s (Single Est: {xgb_risk_single:.2f} s)")

print("\n--- 4. LightGBM (N_EST=50) - Speed Risk Baseline ---")
lgbm_risk_f1, lgbm_risk_std, lgbm_risk_time, lgbm_risk_single = evaluate_f1_macro_cv_timed(lgbm_risk_model, X_train, y_train_cls, cv_cls)
print(f"✅ LGBM CV F1-Macro: {lgbm_risk_f1:.4f} (+/- {lgbm_risk_std:.4f})")
print(f"   Time: {lgbm_risk_time:.2f} s (Single Est: {lgbm_risk_single:.2f} s)")

print("\n--- Final Decision Basis ---")
print(f"XGBoost single training time is {xgb_risk_single:.2f} seconds, confirming it's too slow for the target N_EST.")
print(f"The final script must use LightGBM, as its strongest configuration has a safe single training time of {lgbm_high_single:.2f} seconds.")

--- 1. Logistic Regression (Baseline) ---
✅ LR CV F1-Macro: 0.5087 (+/- 0.0014) | Time: 30.60 s

--- 2. LightGBM (N_EST=200) - Final Performance Target ---
✅ LGBM CV F1-Macro: 0.9217 (+/- 0.0021)
   Time: 133.30 s (Single Est: 44.43 s)

--- 3. XGBoost (N_EST=50) - Speed Risk Assessment ---
✅ XGBoost CV F1-Macro: 0.9022 (+/- 0.0049)
   Time: 375.02 s (Single Est: 125.01 s)

--- 4. LightGBM (N_EST=50) - Speed Risk Baseline ---
✅ LGBM CV F1-Macro: 0.8165 (+/- 0.0038)
   Time: 76.54 s (Single Est: 25.51 s)

--- Final Decision Basis ---
XGBoost single training time is 125.01 seconds, confirming it's too slow for the target N_EST.
The final script must use LightGBM, as its strongest configuration has a safe single training time of 44.43 seconds.


In [49]:
## ### Final Tuning: Step 1: OOF Threshold Optimization (N_est=500)

from lightgbm import LGBMClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
import numpy as np
import time

# --- Lock High Performance Parameters ---
FINAL_CLS_ESTIMATORS = 500 
MAX_DEPTH_FINAL = 15
NUM_LEAVES_FINAL = 70
FINAL_SMOTE_SAMPLING = 0.05
LEARNING_RATE_FINAL = 0.05

cv_threshold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 1. Create OOF Model Pipeline 
# Note: cross_val_predict runs 5 separate fits, which takes a long time
oof_lgbm_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor_reg),
    ('smote', SMOTE(sampling_strategy=FINAL_SMOTE_SAMPLING, random_state=42)),
    ('classifier', LGBMClassifier(
        n_estimators=FINAL_CLS_ESTIMATORS,
        max_depth=MAX_DEPTH_FINAL,
        num_leaves=NUM_LEAVES_FINAL,
        learning_rate=LEARNING_RATE_FINAL,
        random_state=42, n_jobs=1, verbose=-1))
])

print(f"Running Final OOF Threshold Optimization (N_est={FINAL_CLS_ESTIMATORS}, 5-Fold)...")
start_time = time.time()

# 2. Generate OOF Prediction Probabilities
oof_proba = cross_val_predict(
    oof_lgbm_pipeline, X_train, y_train_cls,
    cv=cv_threshold, method="predict_proba", n_jobs=SAFE_N_JOBS
)[:, 1]

end_time = time.time()
print(f"✅ OOF Probability Generation Time: {end_time - start_time:.2f} seconds")


# 3. Find Best Threshold
thresholds = np.linspace(0.001, 0.999, 500)
f1_scores = [f1_score(y_train_cls, (oof_proba >= t).astype(int), average='macro') for t in thresholds]

best_threshold = thresholds[np.argmax(f1_scores)]
best_f1_oof = np.max(f1_scores)

print(f"\n--- Best OOF Threshold Results ---")
print(f"New Model's Best OOF F1-Macro: {best_f1_oof:.4f}")
print(f"New Best Threshold: {best_threshold:.4f}")

# Record the new threshold
NEW_FINAL_CLS_THRESHOLD = best_threshold

Running Final OOF Threshold Optimization (N_est=500, 5-Fold)...
✅ OOF Probability Generation Time: 166.80 seconds

--- Best OOF Threshold Results ---
New Model's Best OOF F1-Macro: 0.9528
New Best Threshold: 0.3310


In [50]:
from lightgbm import LGBMClassifier 
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
import numpy as np
import time

# --- 最终锁定的最佳参数 ---
FINAL_CLS_THRESHOLD = 0.3310      # Optimal Threshold
FINAL_CLS_ESTIMATORS = 500      # Safe performance limit for speed
MAX_DEPTH_FINAL = 15             # Anti-overfitting depth
NUM_LEAVES_FINAL = 70           # Anti-overfitting number of leaves
FINAL_SMOTE_SAMPLING = 0.05     # Latest best SMOTE ratio
LEARNING_RATE_FINAL = 0.05
# 1. Create Final Model Pipeline
final_lgbm_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor_reg),
    ('smote', SMOTE(sampling_strategy=FINAL_SMOTE_SAMPLING, random_state=42)), 
    ('classifier', LGBMClassifier(
        n_estimators=FINAL_CLS_ESTIMATORS, 
        max_depth=MAX_DEPTH_FINAL,        
        num_leaves=NUM_LEAVES_FINAL,      
        random_state=42, n_jobs=1, verbose=-1))
])

# 2. Train on the entire training set (Core time-consuming part of the final script)
print(f"Training Final LightGBM Model (N_est={FINAL_CLS_ESTIMATORS}, SMOTE={FINAL_SMOTE_SAMPLING})...")
start_time = time.time()
# Ensure X_train, y_train_cls are defined
final_lgbm_pipeline.fit(X_train, y_train_cls)
end_time = time.time()
print(f"✅ Final Model Training Time: {end_time - start_time:.2f} seconds")

# 3. Predict on test.csv
# Assuming X_test_cls variable is correctly defined (i.e., contains features of test.csv)
test_proba = final_lgbm_pipeline.predict_proba(X_test_cls)[:, 1]

# 4. Apply the optimized threshold for classification
test_predictions_cls = (test_proba >= FINAL_CLS_THRESHOLD).astype(int)

# 5. Calculate F1 Score on test.csv (Assuming test.csv contains the true 'is_fraud' labels)
if 'is_fraud' in df_test_proc.columns:
    y_test_cls_true = df_test_proc['is_fraud']
    final_f1_on_test = f1_score(y_test_cls_true, test_predictions_cls, average='macro')
    
    print(f"\n--- Final Performance Report (Test Set) ---")
    print(f"Final LGBM Model F1-Macro on test.csv: {final_f1_on_test:.4f}")
    
    # Report gap to target
    target_f1 = 0.97
    difference = target_f1 - final_f1_on_test
    
    if final_f1_on_test >= target_f1:
        print(f"🎉🎉 Part III Goal Achieved! Final F1 Score: {final_f1_on_test:.4f}.")
    else:
        print(f"⚠️ **Final Gap:** Remaining difference to target F1={target_f1:.4f}: **{difference:.4f}**.")
        print(f"✅ Final Conclusion: This is the best generalizing F1 Score under the strict 2-minute time limit.")
FINAL_CLS_PREDICTIONS = test_predictions_cls


Training Final LightGBM Model (N_est=500, SMOTE=0.05)...
✅ Final Model Training Time: 54.06 seconds


d:\github\Ml\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



--- Final Performance Report (Test Set) ---
Final LGBM Model F1-Macro on test.csv: 0.9107
⚠️ **Final Gap:** Remaining difference to target F1=0.9700: **0.0593**.
✅ Final Conclusion: This is the best generalizing F1 Score under the strict 2-minute time limit.
